In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Example of loading data into a pandas DataFrame (replace with your actual data loading)
df = pd.read_csv('temp/test.csv')

# Display the first few rows of the dataframe
df.head()


,Day,Month,Year,Hour,Minute,Night,Holiday,Block,lat,long,...,StreetType-CR,StreetType-OTHER,StreetType-AL,StreetType-WK,StreetType-RW,StreetType-I-80,Season-Spring,Season-Winter,Season-Fall,Season-Summer
0,13,5,12,23,53,1,0,0,0.313018,-0.123732,...,0,0,0,0,0,0,1,0,0,0
1,13,5,12,23,53,1,0,0,0.313018,-0.123732,...,0,0,0,0,0,0,1,0,0,0
2,13,5,12,23,33,1,0,0,1.381346,-0.063274,...,0,0,0,0,0,0,1,0,0,0
3,13,5,12,23,30,1,0,16,1.400312,-0.167381,...,0,0,0,0,0,0,1,0,0,0
4,13,5,12,23,30,1,0,2,0.186493,-0.631787,...,0,0,0,0,0,0,1,0,0,0


In [3]:
# Define features (X) and target (y)
X = df.drop('Category', axis=1)  # Features: all columns except 'Category'
y = df['Category']  # Target variable: 'Category'

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

In [5]:
# Fitting the model
rf_classifier.fit(X_train, y_train)

: 

In [ ]:
# Predict on the test data
y_pred = rf_classifier.predict(X_test)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")